In [3]:
!pip install nltk gensim

In [45]:
"""import json
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


def predict_NuExtract(model, tokenizer, text, schema, example=["","",""]):
    schema = json.dumps(json.loads(schema), indent=4)
    input_llm =  "<|input|>\n### Template:\n" +  schema + "\n"
    for i in example:
      if i != "":
          input_llm += "### Example:\n"+ json.dumps(json.loads(i), indent=4)+"\n"
    
    input_llm +=  "### Text:\n"+text +"\n<|output|>\n"
    input_ids = tokenizer(input_llm, return_tensors="pt", truncation=True, max_length=4000).to("cuda")

    output = tokenizer.decode(model.generate(**input_ids)[0], skip_special_tokens=True)
    return output.split("<|output|>")[1].split("<|end-output|>")[0]


model = AutoModelForCausalLM.from_pretrained("numind/NuExtract", trust_remote_code=True, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("numind/NuExtract", trust_remote_code=True)

model.to("cuda")

model.eval()

text = """"""We introduce Mistral 7B, a 7–billion-parameter language model engineered for
superior performance and efficiency. Mistral 7B outperforms the best open 13B
model (Llama 2) across all evaluated benchmarks, and the best released 34B
model (Llama 1) in reasoning, mathematics, and code generation. Our model
leverages grouped-query attention (GQA) for faster inference, coupled with sliding
window attention (SWA) to effectively handle sequences of arbitrary length with a
reduced inference cost. We also provide a model fine-tuned to follow instructions,
Mistral 7B – Instruct, that surpasses Llama 2 13B – chat model both on human and
automated benchmarks. Our models are released under the Apache 2.0 license.
Code: https://github.com/mistralai/mistral-src
Webpage: https://mistral.ai/news/announcing-mistral-7b/""""""

schema = """"""{
    "Model": {
        "Name": "",
        "Number of parameters": "",
        "Number of token": "",
        "Architecture": []
    },
    "Usage": {
        "Use case": [],
        "Licence": ""
    }
}""""""

prediction = predict_NuExtract(model, tokenizer, text, schema, example=["","",""])
print(prediction)
"""

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does no

OSError: [Errno 28] No space left on device

In [4]:
import pandas as pd
import time
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords
from groq import Groq
import os
from dotenv import load_dotenv
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oskarroeske/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# EDA

In [35]:
df_reports = pd.read_csv("preprocessed_reports.csv")
df_paragraphs = pd.read_csv("preprocessed_paragraphs.csv")

In [42]:
df_paragraphs_testing = df_paragraphs[0:20]

In [43]:
df_paragraphs_testing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  20 non-null     int64 
 1   ID          20 non-null     int64 
 2   paragraphs  20 non-null     object
dtypes: int64(2), object(1)
memory usage: 612.0+ bytes


# FINBERT

# KeyBERT

In [ ]:
import pandas as pd
from keybert import KeyBERT

# Initialize KeyBERT model
kw_model = KeyBERT()

# Function to extract keywords or key phrases
def extract_keywords(text, ngram_range=(1, 4), top_n=5):
    keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=ngram_range, stop_words='english', top_n=top_n)
    # Return just the keywords (without similarity scores)
    return [kw for kw in keywords]

# LLM Argument Mining

- Provide List of pre-defined arguments

In [38]:
def find_arguments(text, api_key):
    client = Groq(api_key=api_key)

    instructions = """I will give you a financial text, and here are the instructions:
-Collect the main arguments and assign them to a category.
-Categories must be short, generic, and a maximum of 3 words.
-Check if the argument fits any of the existing categories.
-Create a new category if no match is found.
-Update the existing categories afterwards
-Ignore Target Prices/Recommendation as arguments.
-Don't include brand names in argument
-Classify arguments as positive/negative/neutral based on wording, but only return the raw categories, excluding any words that suggest sentiment (e.g., 'Cost Management' instead of 'Cost Management Growth').
-Return the arguments and sentiment as a dictionary (e.g., {'Category 1': 'positive', 'Category 2': 'negative',...).
-the result should always be in one line and every key and value should be within quotation marks
-Return only the dictionary, no other text."""
    final_prompt = instructions + "\n Text: " +text

    response = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "Response must be correct formatted json. No additional text must be provided.",
            },
            {
                "role": "user",
                "content": final_prompt,
            }
        ],
        temperature=0,
        model="llama3-70b-8192",
    )

    return response.choices[0].message.content

In [39]:
# Define a helper function that adds a delay
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")

def find_arguments_with_delay(commentary):
    result = find_arguments(commentary,api_key)
    print(result)
    time.sleep(2)  # 2.5-second delay
    return result


In [45]:
df_paragraphs_testing["provided arguments"] = df_paragraphs_testing["paragraphs"].apply(find_arguments_with_delay)

{"Valuation": "negative", "Advertising": "positive", "EBIT Margins": "positive", "Business Unit": "neutral"}
{"Earnings Power": "positive", "Business Dynamics": "positive", "Upside Potential": "positive", "Earnings Estimates": "positive"}
{"Fulfillment Capacity": "positive", "Investment Cycle": "positive", "Guidance Challenges": "negative", "Long Term Growth": "positive"}
{"Stock Valuation": "neutral", "Analyst Credibility": "negative", "Business Transformation": "positive", "Cost Management": "neutral", "Profitability": "neutral"}
{"Disclosure": "positive", "Business Complexity": "neutral", "Profitability": "positive"}
{"Reported Facts": "neutral", "Assumptions": "negative", "Segmentation": "neutral"}
{"Gross Margin": "positive", "Growth Assumption": "positive", "Holding Company Discount": "negative", "Party GMV": "neutral", "Take Rate": "positive"}
{"Investment Mode": "positive", "International GMV": "neutral", "Losses": "negative"}
{"Market Performance": "neutral", "Government Regul

/var/folders/2h/tyzkflxs465698k6z8l3ktj00000gn/T/ipykernel_43345/1854909684.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_paragraphs_testing["provided arguments"] = df_paragraphs_testing["paragraphs"].apply(find_arguments_with_delay)


In [46]:
df_paragraphs_testing.head()

,Unnamed: 0,ID,paragraphs,provided arguments
0,0,1,BOTTOM LINE We and almost every other investor...,"{""Valuation"": ""negative"", ""Advertising"": ""posi..."
1,1,1,Based on our view that there is massive upside...,"{""Earnings Power"": ""positive"", ""Business Dynam..."
2,2,1,We would highlight that this is not a shortter...,"{""Fulfillment Capacity"": ""positive"", ""Investme..."
3,3,1,Everybody who is using a SOTP model for AMZN i...,"{""Stock Valuation"": ""neutral"", ""Analyst Credib..."
4,4,1,"SOTP debates aside, we would all concur that m...","{""Disclosure"": ""positive"", ""Business Complexit..."


In [47]:
df_paragraphs_testing.to_csv("arguments_testing.csv")

In [36]:
#df.to_csv("with_dictionaries.csv")

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Company                   2 non-null      object        
 1   Short Name                2 non-null      object        
 2   Recommendation            2 non-null      object        
 3   Target Price              2 non-null      object        
 4   Date                      2 non-null      datetime64[ns]
 5   Commentary                2 non-null      object        
 6   Text ID                   2 non-null      int64         
 7   Date next Day             2 non-null      datetime64[ns]
 8   Date after 3 Months       2 non-null      datetime64[ns]
 9   Date after 6 Months       2 non-null      datetime64[ns]
 10  Date after 9 Months       2 non-null      datetime64[ns]
 11  Date after 12 Months      2 non-null      datetime64[ns]
 12  Start Price               

In [38]:
import pandas as pd
import json
from collections import Counter, defaultdict

# Initialize a dictionary to hold counts for each key and sentiment
sentiment_counts = defaultdict(lambda: Counter({'positive': 0, 'neutral': 0, 'negative': 0}))

# Loop through each entry in the provided_arguments column
for entry in df_sentences["Provided Arguments"]:
    # Ensure the entry is a dictionary
    if isinstance(entry, str):
        try:
            # Convert string to dictionary if it's in JSON-like format
            arguments_dict = json.loads(entry.replace("'", '"'))  # Replace single quotes with double quotes for JSON
        except json.JSONDecodeError:
            continue  # Skip any rows that can't be parsed as dictionaries
    elif isinstance(entry, dict):
        arguments_dict = entry
    else:
        continue  # Skip if entry is not a dictionary or parseable string

    # Update counts for each key and sentiment
    for key, sentiment in arguments_dict.items():
        sentiment_counts[key][sentiment] += 1

# Convert the result to a DataFrame for easier viewing
sentiment_overview_df = pd.DataFrame(sentiment_counts).T
sentiment_overview_df.columns = ["positive", "neutral", "negative"]
sentiment_overview_df.index.name = "Argument"

# Display the overview DataFrame
sentiment_overview_df.reset_index(inplace=True)
sentiment_overview_df.sort_values(by=["positive","negative","neutral"],ascending=False)


,Argument,positive,neutral,negative
0,Revenue Growth,4,0,0
1,Financial Performance,3,1,1
8,Valuation,1,1,0
3,Revenue,1,0,0
6,Growth Expectations,1,0,0
7,Revenue Outlook,1,0,0
9,Pricing Power,1,0,0
10,Growth Potential,1,0,0
11,Marketing Efforts,1,0,0
12,Patient Growth,1,0,0


# Check amount of intensifiers

In [39]:

# Basic Intensifiers
basic_intensifiers = [
    "very", "really", "quite", "so", "too", "totally", "absolutely",
    "completely", "utterly", "entirely", "truly", "fully", "genuinely", "surely"
]

# Strong Intensifiers (includes previous stronger, superlative, and "all/full" intensifiers)
strong_intensifiers = [
    "extremely", "incredibly", "highly", "immensely", "vastly", "remarkably", 
    "tremendously", "phenomenally", "exceptionally", "noticeably", "supremely", 
    "powerfully", "acutely", "enormously", "greatly", "immeasurably", "infinitely",
    "wildly", "lavishly", "ludicrously", "fiercely", "ferociously", "intensely",
    "extraordinarily", "overwhelmingly", "staggeringly", "excessively", "insanely", 
    "unbelievably", "inconceivably", "ridiculously", "crazily", "all-out", 
    "all-around", "full-on", "fully", "flat-out"
]

# Formal/Emotive Intensifiers (combines emotive and formal language for a more academic or dramatic tone)
formal_emotive_intensifiers = [
    "profoundly", "considerably", "significantly", "markedly", "appreciably", 
    "notably", "exceedingly", "far", "deeply", "awfully", "horribly", "dreadfully", 
    "frightfully", "painfully", "absurdly", "scandalously", "outrageously", 
    "shockingly", "horrendously", "terribly", "hideously", "grossly"
]


In [40]:
import re

# Function to count intensifiers in a single text
def count_intensifiers(text, intensifiers):
    # Convert to lowercase to make it case-insensitive
    words = re.findall(r'\b\w+\b', text.lower())
    return sum(words.count(word) for word in intensifiers)

In [41]:
# Apply the function to the "Sentence" column of the DataFrame
df_sentences["Basic Intensifier"] = df_sentences["Sentence"].apply(count_intensifiers, intensifiers=basic_intensifiers)
df_sentences["Strong Intensifier"] = df_sentences["Sentence"].apply(count_intensifiers, intensifiers=strong_intensifiers)
df_sentences["Formal Intensifier"] = df_sentences["Sentence"].apply(count_intensifiers, intensifiers=formal_emotive_intensifiers)

In [42]:
df_sentences

,Text ID,Sentence ID,Sentence,Provided Arguments,Basic Intensifier,Strong Intensifier,Formal Intensifier
0,1,1,Q1 Results: Jazz Pharmaceuticals reported stro...,"{""Revenue Growth"": ""positive"", ""Financial Perf...",0,0,0
1,1,2,The GAAP Net Loss $93 million vs. GAAP Net Inc...,"{""Financial Performance"": ""negative"", ""License...",0,0,0
2,1,3,Adjusted Net Income $101 million vs. $84 milli...,"{""Financial Performance"": ""positive""}",0,0,0
3,1,4,These figures slightly ahead Goldman Stanley e...,"{""Revenue"": ""positive"", ""Financial Performance...",0,0,0
4,1,5,■ FY 14 Guidance: The Company reaffirmed FY 14...,"{""Revenue Guidance"": ""neutral"", ""Sales Perform...",0,0,0
5,1,6,While believe market already priced expectatio...,"{""Growth Expectations"": ""positive"", ""Revenue O...",0,0,0
6,1,7,■ Catalysts: 1) Possible price increases Xyrem...,"{""Pricing Power"": ""positive"", ""Growth Potentia...",0,0,0
7,1,8,2) Launch new marketing campaigns Xyrem – Jazz...,"{""Marketing Efforts"": ""positive"", ""Patient Gro...",0,0,0
8,1,9,3) Settlement Roxane lawsuit – We believe deci...,"{""Litigation"": ""positive"", ""Product Launch"": ""...",0,0,0
9,1,10,Current market expectations point generics FY ...,"{""Market Expectations"": ""neutral"", ""Sales Grow...",0,0,0


In [43]:
df["Commentary"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 2 entries, 0 to 1
Series name: Commentary
Non-Null Count  Dtype 
--------------  ----- 
2 non-null      object
dtypes: object(1)
memory usage: 148.0+ bytes


In [44]:
print(df[df["Formal Intensifier"] > 1])

KeyError: 'Formal Intensifier'